In [54]:
import pandas as pd
import numpy as np
import requests,os,json
import time,openpyxl

from groq import Groq

from dotenv import load_dotenv


load_dotenv() # Loads variables from .env in the current directory

API_KEY = os.getenv("API_KEY")


client = Groq(

    api_key=API_KEY,

)

In [ ]:
#getting data by scraping Fortune 100 companies data in 
# Json format (fortune_100_companies_json.txt) and reading it

with open("fortune_100_companies_json.txt", "r") as f:
        company_data = json.load(f)

In [ ]:
#cleaning the Json data and getting only the releavnt data

company_name=[]
industry=[]
hq=[]
annual_revenue=[]
employee_data=[]
for i in range (100):
  company_name.append(company_data[i]['name'])
  industry.append(company_data[i]['data']['Industry'])
  hq.append(company_data[i]['data']['Headquarters City']+", "+company_data[i]['data']['State'])
  annual_revenue.append(company_data[i]['data']['Revenues ($M)'])
  employee_data.append(company_data[i]['data']['Employees'])

In [98]:
dict = {'Company Name': company_name, 'Industry': industry, 'Headquarters Location':hq, 'Estimated Annual Revenue':annual_revenue, \
        'Number of Employees':employee_data}

df = pd.DataFrame(dict)
df

,Company Name,Industry,Headquarters Location,Estimated Annual Revenue,Number of Employees
0,Walmart,General Merchandisers,"Bentonville, Arkansas","$680,985","2,100,000"
1,Amazon,Internet Services and Retailing,"Seattle, Washington","$637,959","1,556,000"
2,UnitedHealth Group,Health Care: Insurance and Managed Care,"Eden Prairie, Minnesota","$400,278","400,000"
3,Apple,"Computers, Office Equipment","Cupertino, California","$391,035","164,000"
4,CVS Health,Health Care: Pharmacy and Other Services,"Woonsocket, Rhode Island","$372,809","259,500"
...,...,...,...,...,...
95,General Dynamics,Aerospace & Defense,"Reston, Virginia","$47,716","117,000"
96,Coca-Cola,Beverages,"Atlanta, Georgia","$47,061","69,700"
97,TIAA,"Insurance: Life, Health (Mutual)","New York, New York","$46,945.8","15,623"
98,Travelers,Insurance: Property and Casualty (Stock),"New York, New York","$46,423","34,000"


In [ ]:
data=[]
for company in df['Company Name']:
  
  prompt = f"""
Given a company name, enrich its profile with public data on employee count, recent funding (last 12 months), key technologies, and AI/SaaS signals. Use web_search or browse_page on credible sources (e.g., Crunchbase, LinkedIn, company site, TechCrunch). Return a JSON object with:

- `company_name`: {company}
- `recent_funding`: Funding rounds from 09/25/2024 to 09/25/2025 (round type, amount, date MM/DD/YYYY, lead investor).Find the recent funding details if available. Bulleted list string or "No funding in last 12 months." or "No recent funding found. "
- `key_technologies`: 3-5 technologies/software used (e.g., "Python, AWS, Salesforce"). Comma-separated list.
- `ai_saas_signals`: 3-5 signals of AI/SaaS interest (e.g., press releases, job postings, news, last 12 months). Numbered list string with source and date, or "No recent signals found."


Ethical: Use only public data, respect robots.txt. Limit tool calls to 5.

**Output Format** (JSON):
{{
  "company_name": "string",
  "recent_funding": "string",
  "key_technologies": "string",
  "ai_saas_signals": "string"
}}

"""
  chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": prompt,
        }
    ],

    model="meta-llama/llama-4-maverick-17b-128e-instruct",
    temperature=0.1,
    response_format={"type": "json_object"}
  )
  data_dict = json.loads(chat_completion.choices[0].message.content)
  data.append(data_dict)


  print(chat_completion.choices[0].message.content)
  time.sleep(10) #keeping the api hits within free tier 






{
  "company_name": "Walmart",
  "recent_funding": "No recent funding found.",
  "key_technologies": "AWS, Salesforce, Java, Oracle, Microsoft Azure",
  "ai_saas_signals": "1. Walmart announced plans to integrate AI-powered chatbots into its customer service platform (TechCrunch, 08/15/2024). 2. Walmart launched a new SaaS-based supply chain management platform (Supply Chain Dive, 07/20/2024). 3. Walmart is hiring for AI/ML engineering roles (LinkedIn, 09/01/2024)."
}
{
  "company_name": "Amazon",
  "recent_funding": "No recent funding found. ",
  "key_technologies": "AWS, Python, Java, Kubernetes, TensorFlow",
  "ai_saas_signals": "1. Amazon announced plans to invest $15 billion in AI infrastructure (Reuters, 08/15/2024). 2. Amazon SageMaker now supports AutoML for time-series forecasting (AWS Blog, 07/20/2024). 3. Amazon launched a new AI-powered coding assistant, CodeWhisperer (TechCrunch, 06/22/2024)."
}
{
  "company_name": "UnitedHealth Group",
  "recent_funding": "No recent fundi

In [49]:
ai_data=pd.DataFrame(data)
ai_data

,company_name,recent_funding,key_technologies,ai_saas_signals
0,Walmart,No recent funding found.,"AWS, Salesforce, Java, Oracle, Microsoft Azure",1. Walmart announced plans to integrate AI-pow...
1,Amazon,No recent funding found.,"AWS, Python, Java, Kubernetes, TensorFlow",1. Amazon announced plans to invest $15 billio...
2,UnitedHealth Group,No recent funding found.,"Python, AWS, Epic Systems, Oracle, Microsoft A...",1. UnitedHealth Group's Optum division launche...
3,Apple,No recent funding found.,"Swift, iOS, macOS, AWS, Kubernetes",1. Apple announces plans to integrate AI into ...
4,CVS Health,No recent funding found.,"Python, AWS, Epic Systems, Salesforce, Oracle",1. CVS Health announced plans to leverage AI f...
...,...,...,...,...
95,General Dynamics,No recent funding found.,"C++, Java, Python, AWS, Azure",1. General Dynamics touts AI-infused Gulfstrea...
96,Coca-Cola,No recent funding found.,"SAP, Oracle, Salesforce, AWS",1. Coca-Cola announced a partnership with Micr...
97,TIAA,No recent funding found.,"Python, AWS, Salesforce, Java, Oracle",1. TIAA launches AI-powered retirement plannin...
98,Travelers,No recent funding found.,"Python, AWS, Salesforce, Java, Oracle",1. Travelers Companies explores AI-powered ins...


In [102]:
merged_df = pd.merge(df, ai_data, left_on='Company Name', right_on='company_name', how='left')
merged_df

,Company Name,Industry,Headquarters Location,Estimated Annual Revenue,Number of Employees,company_name,recent_funding,key_technologies,ai_saas_signals
0,Walmart,General Merchandisers,"Bentonville, Arkansas","$680,985","2,100,000",Walmart,No recent funding found.,"AWS, Salesforce, Java, Oracle, Microsoft Azure",1. Walmart announced plans to integrate AI-pow...
1,Amazon,Internet Services and Retailing,"Seattle, Washington","$637,959","1,556,000",Amazon,No recent funding found.,"AWS, Python, Java, Kubernetes, TensorFlow",1. Amazon announced plans to invest $15 billio...
2,UnitedHealth Group,Health Care: Insurance and Managed Care,"Eden Prairie, Minnesota","$400,278","400,000",UnitedHealth Group,No recent funding found.,"Python, AWS, Epic Systems, Oracle, Microsoft A...",1. UnitedHealth Group's Optum division launche...
3,Apple,"Computers, Office Equipment","Cupertino, California","$391,035","164,000",Apple,No recent funding found.,"Swift, iOS, macOS, AWS, Kubernetes",1. Apple announces plans to integrate AI into ...
4,CVS Health,Health Care: Pharmacy and Other Services,"Woonsocket, Rhode Island","$372,809","259,500",CVS Health,No recent funding found.,"Python, AWS, Epic Systems, Salesforce, Oracle",1. CVS Health announced plans to leverage AI f...
...,...,...,...,...,...,...,...,...,...
95,General Dynamics,Aerospace & Defense,"Reston, Virginia","$47,716","117,000",General Dynamics,No recent funding found.,"C++, Java, Python, AWS, Azure",1. General Dynamics touts AI-infused Gulfstrea...
96,Coca-Cola,Beverages,"Atlanta, Georgia","$47,061","69,700",Coca-Cola,No recent funding found.,"SAP, Oracle, Salesforce, AWS",1. Coca-Cola announced a partnership with Micr...
97,TIAA,"Insurance: Life, Health (Mutual)","New York, New York","$46,945.8","15,623",TIAA,No recent funding found.,"Python, AWS, Salesforce, Java, Oracle",1. TIAA launches AI-powered retirement plannin...
98,Travelers,Insurance: Property and Casualty (Stock),"New York, New York","$46,423","34,000",Travelers,No recent funding found.,"Python, AWS, Salesforce, Java, Oracle",1. Travelers Companies explores AI-powered ins...


In [103]:
merged_df.drop('company_name', axis=1, inplace=True)
merged_df.rename(columns={'recent_funding': 'Recent Funding or Investments'\
                          ,'key_technologies':'Key Technologies and Software Used'\
                            ,'ai_saas_signals':'Recent Public Signals of Interest in AI or SaaS'}\
                              , inplace=True)



In [107]:
# saving the output to excel file
merged_df.to_excel("fortune_100_final_sheet.xlsx", index=False)
